# system_utils 모듈 예제

시스템 유틸리티 - 로깅 설정 및 관리

In [1]:
import logging
import warnings
from system_utils import get_logger, setup_logger

2025-12-04 07:34:46, [INFO], [logger.py], Log files created:
2025-12-04 07:34:46, [INFO], [logger.py],   Main log: logs\log_20251204_073446_info.log
2025-12-04 07:34:46, [INFO], [logger.py],   Debug log: logs\log_20251204_073446_debug.log


## 1. 기본 사용

전역 로거를 가져와서 사용합니다.

In [2]:
# 전역 로거 가져오기
logger = get_logger()

# 다양한 레벨의 로그 출력
logger.debug("Debug message - debug.log에만 저장")
logger.info("Info message - 콘솔 + info.log")
logger.warning("Warning message - 콘솔 + info.log")
logger.error("Error message - 콘솔 + info.log")

2025-12-04 07:34:46, [INFO], [3804881514.py], Info message - 콘솔 + info.log
2025-12-04 07:34:46, [WARN], [3804881514.py], Warning message - 콘솔 + info.log
2025-12-04 07:34:46, [ERROR], [3804881514.py], Error message - 콘솔 + info.log


## 2. 로그 레벨 이해

| 레벨 | 숫자 | 콘솔 | info.log | debug.log |
|------|------|------|----------|----------|
| DEBUG | 10 | ❌ | ❌ | ✅ |
| FUTUREWARN | 19 | ❌ | ❌ | ✅ |
| INFO | 20 | ✅ | ✅ | ❌ |
| WARNING | 30 | ✅ | ✅ | ❌ |
| ERROR | 40 | ✅ | ✅ | ❌ |

In [3]:
# 로그 레벨 확인
print("Standard log levels:")
print(f"  DEBUG: {logging.DEBUG}")
print(f"  INFO: {logging.INFO}")
print(f"  WARNING: {logging.WARNING}")
print(f"  ERROR: {logging.ERROR}")
print(f"\nCustom levels:")
print(f"  FUTUREWARN: 19")

Standard log levels:
  DEBUG: 10
  INFO: 20
  ERROR: 40

Custom levels:
  FUTUREWARN: 19


## 3. FutureWarning 처리

Python FutureWarning이 자동으로 로거로 리다이렉트됩니다.

In [4]:
# FutureWarning 발생시키기
# 콘솔에는 요약만, debug.log에는 상세 정보 저장
warnings.warn("This feature will be deprecated in future versions", FutureWarning)

2025-12-04 07:34:46, [WARN], [logger.py], FutureWarning in C:\Users\coldm\AppData\Local\Temp\ipykernel_46648\1874229810.py:3


In [5]:
# 일반 Warning
warnings.warn("This is a general warning", UserWarning)

2025-12-04 07:34:46, [WARN], [logger.py], C:\Users\coldm\AppData\Local\Temp\ipykernel_46648\3096136593.py:2: UserWarning: This is a general warning
  warnings.warn("This is a general warning", UserWarning)


## 4. 커스텀 로거 생성

다른 설정의 로거가 필요한 경우 새로 생성할 수 있습니다.

In [6]:
# 콘솔 출력 없이 파일만 저장
file_logger = setup_logger("FileOnly", console=False, log_dir="logs_file_only")
file_logger.info("This will only be saved to file")

In [7]:
# 파일 저장 없이 콘솔만
console_logger = setup_logger("ConsoleOnly", file=False)
console_logger.info("This will only be printed to console")

2025-12-04 07:34:46, [INFO], [2115674886.py], This will only be printed to console


## 5. 활용 예시

### 5.1 시뮬레이션 로깅

In [8]:
logger = get_logger()

def simulate_step(step, data):
    """시뮬레이션 스텝 예시"""
    logger.debug(f"Step {step}: Raw data = {data}")
    
    # 처리
    result = sum(data) / len(data)
    
    if result > 50:
        logger.warning(f"Step {step}: Result {result:.2f} exceeds threshold")
    else:
        logger.info(f"Step {step}: Result = {result:.2f}")
    
    return result

# 시뮬레이션 실행
for i in range(5):
    data = [10 + i*10, 20 + i*10, 30 + i*10]
    simulate_step(i, data)

2025-12-04 07:34:46, [INFO], [361651509.py], Step 0: Result = 20.00
2025-12-04 07:34:46, [INFO], [361651509.py], Step 1: Result = 30.00
2025-12-04 07:34:46, [INFO], [361651509.py], Step 2: Result = 40.00
2025-12-04 07:34:46, [INFO], [361651509.py], Step 3: Result = 50.00
2025-12-04 07:34:46, [WARN], [361651509.py], Step 4: Result 60.00 exceeds threshold


### 5.2 여러 모듈에서 공유

In [9]:
# 모듈 A 시뮬레이션
class ModuleA:
    def __init__(self):
        self.logger = get_logger()
        self.logger.info("ModuleA initialized")
    
    def process(self):
        self.logger.info("ModuleA processing...")

# 모듈 B 시뮬레이션
class ModuleB:
    def __init__(self):
        self.logger = get_logger()
        self.logger.info("ModuleB initialized")
    
    def process(self):
        self.logger.info("ModuleB processing...")

# 같은 로거 공유
a = ModuleA()
b = ModuleB()
a.process()
b.process()

2025-12-04 07:34:46, [INFO], [1826650547.py], ModuleA initialized
2025-12-04 07:34:46, [INFO], [1826650547.py], ModuleB initialized
2025-12-04 07:34:46, [INFO], [1826650547.py], ModuleA processing...
2025-12-04 07:34:46, [INFO], [1826650547.py], ModuleB processing...


### 5.3 에러 핸들링과 로깅

In [10]:
logger = get_logger()

def safe_divide(a, b):
    """안전한 나눗셈 with 로깅"""
    logger.debug(f"Dividing {a} by {b}")
    try:
        result = a / b
        logger.info(f"Division result: {result}")
        return result
    except ZeroDivisionError:
        logger.error("Division by zero!")
        return None

# 정상 케이스
safe_divide(10, 2)

# 에러 케이스
safe_divide(10, 0)

2025-12-04 07:34:47, [INFO], [2222522779.py], Division result: 5.0
2025-12-04 07:34:47, [ERROR], [2222522779.py], Division by zero!


## 6. 로그 파일 확인

로그 파일은 `logs/` 폴더에 생성됩니다:
- `log_YYYYMMDD_HHMMSS_info.log` - INFO 이상 로그
- `log_YYYYMMDD_HHMMSS_debug.log` - DEBUG ~ FUTUREWARN 로그

In [11]:
import os

# 로그 디렉토리 확인
log_dir = "logs"
if os.path.exists(log_dir):
    files = os.listdir(log_dir)
    print(f"Log files in '{log_dir}/':\n")
    for f in sorted(files):
        filepath = os.path.join(log_dir, f)
        size = os.path.getsize(filepath)
        print(f"  {f} ({size} bytes)")
else:
    print(f"Log directory '{log_dir}' not found")

Log files in 'logs/':

  log_20251204_073446_debug.log (904 bytes)
  log_20251204_073446_info.log (1588 bytes)
